In [1]:
import os
import sys
import json
import uuid
import datetime

from mlgeneric.mlsftp import *
from mlgeneric.classification import *
from mlgeneric.regression import *

from efficient_apriori import apriori

C:\Users\doliy\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


<Figure size 960x640 with 0 Axes>

In [2]:
df=pd.read_csv(r"dataset\sample_transactions.txt",delimiter=',')
print(df.shape)
df.head()

(92, 17)


,OperationName,RequestTimestamp,RequestSiteId,RequestSalePointId,RequestBasketIdTypeId,RequestBasketValue,ResponseProcessingTimestamp,RequestMessageId,RequestBasketId,ResponseMessageId,RequestTransmitAttempt,ResponseRgBasketId,ResponseCode,RequestNumberBasketItems,ResponseFinancialTimestamp,RequestBasketJsonString,LoyaltyId
0,RegisterBasket,2018-11-27 14:13:27 +00:00,7333,5,NaN,14.60000,2018-11-27 15:25:56 +00:00,1181127141327733305,1-73335271658,1525560091284408562,1,cf31c58d05ca41bdaa500003caf21bf1,10100,5,2018-11-27 15:25:56 +00:00,"{""id"":""73335271658"",""curr"":""null"",""val"":14.6,""...",NaN
1,RegisterBasket,2018-11-27 16:35:31 +00:00,5775,2,NaN,2.54000,2018-11-27 16:25:35 +00:00,1181127163531577502,1-577522888398,1625359427184208983,1,1d385ce056e1463a88510003cd0b191a,10100,3,2018-11-27 16:25:35 +00:00,"{""id"":""577522888398"",""curr"":""null"",""val"":2.54,...",NaN
2,RegisterBasket,2018-11-27 22:34:50 +00:00,7392,7,NaN,6.35000,2018-11-27 22:43:46 +00:00,1181127223450739207,7392753009,224346798075297527,1,b9083214cf6c4593b58800041bc6a5ef,10100,4,2018-11-27 22:43:46 +00:00,"{""id"":""7392753009"",""curr"":""null"",""val"":6.35,""i...",NaN
3,RegisterBasket,2018-11-27 11:52:12 +00:00,8487,3,NaN,18.63000,2018-11-27 11:52:30 +00:00,1181127115212848703,84873530474,1152301406298277102,1,7da158b1c1d14fdf8b8c00042613d18c,10100,9,2018-11-27 11:52:30 +00:00,"{""id"":""84873530474"",""curr"":""null"",""val"":18.63,...",NaN
4,RegisterBasket,2018-11-27 14:20:49 +00:00,7868,1,NaN,42.99000,2018-11-27 14:27:34 +00:00,1181127142049786801,786812414557,1427344860262836840,1,ce2c0ef55f4c47d1b15e0004397e8a9b,10100,4,2018-11-27 14:27:34 +00:00,"{""id"":""786812414557"",""curr"":""null"",""val"":42.99...",NaN


In [3]:
df=df[["ResponseRgBasketId","RequestBasketJsonString"]]

In [4]:
df["RequestBasketJsonString"]

0     {"id":"73335271658","curr":"null","val":14.6,"...
1     {"id":"577522888398","curr":"null","val":2.54,...
2     {"id":"7392753009","curr":"null","val":6.35,"i...
3     {"id":"84873530474","curr":"null","val":18.63,...
4     {"id":"786812414557","curr":"null","val":42.99...
                            ...                        
87    {"id":"80495233101","curr":"null","val":2.0,"i...
88    {"id":"980871386930","curr":"null","val":4.5,"...
89    {"id":"51606159417","curr":"null","val":1.5,"i...
90    {"id":"980871386931","curr":"null","val":4.5,"...
91    {"id":"51606159418","curr":"null","val":1.5,"i...
Name: RequestBasketJsonString, Length: 92, dtype: object

In [5]:
data = []
Group_id = 0
for i in range(0,len(df[["ResponseRgBasketId"]])):
    response = json.loads(df["RequestBasketJsonString"].iloc[i])
    Group_id += 1 
    #print(response)
    #print(response['id'])
    i = 0
    for nest in response['items']:
        i += 1
        if i ==50:
            break
        data.append([Group_id,"Inp_item_"+str(i),str(nest['b'])])
data

[[1, 'Inp_item_1', '5017838815183'],
 [1, 'Inp_item_2', '5017838815183'],
 [1, 'Inp_item_3', '5000128725477'],
 [1, 'Inp_item_4', '5000128725477'],
 [1, 'Inp_item_5', '5010115926985'],
 [2, 'Inp_item_1', '8710732226907'],
 [2, 'Inp_item_2', '5010891011868'],
 [2, 'Inp_item_3', '5015552000151'],
 [3, 'Inp_item_1', '7622300754136'],
 [3, 'Inp_item_2', '5000128797436'],
 [3, 'Inp_item_3', '5010718402909'],
 [3, 'Inp_item_4', '5038862320108'],
 [4, 'Inp_item_1', '5060107333970'],
 [4, 'Inp_item_2', '5000205039862'],
 [4, 'Inp_item_3', '5000205034546'],
 [4, 'Inp_item_4', '5010228013213'],
 [4, 'Inp_item_5', '8712566328352'],
 [4, 'Inp_item_6', '4062300011847'],
 [4, 'Inp_item_7', '5060107331365'],
 [4, 'Inp_item_8', '5010102240537'],
 [4, 'Inp_item_9', '5000128889889'],
 [5, 'Inp_item_1', '6191514300312'],
 [5, 'Inp_item_2', '5000128715331'],
 [5, 'Inp_item_3', '5000128871457'],
 [5, 'Inp_item_4', '5000128271165'],
 [6, 'Inp_item_1', '9770307268922'],
 [6, 'Inp_item_2', '5000128861069'],
 

In [6]:
# Saving the file into the home directory
df_transactions_long = pd.DataFrame(data, columns = ['Group_id','Obs' ,'Item_barcode'])
df_transactions_long = df_transactions_long.astype(str)
#l=list(df_transactions_long.index)
df_transactions_long

,Group_id,Obs,Item_barcode
0,1,Inp_item_1,5017838815183
1,1,Inp_item_2,5017838815183
2,1,Inp_item_3,5000128725477
3,1,Inp_item_4,5000128725477
4,1,Inp_item_5,5010115926985
...,...,...,...
259,90,Inp_item_1,0000050457243
260,91,Inp_item_1,5000128677127
261,91,Inp_item_2,7622210702005
262,91,Inp_item_3,5022313997995


In [7]:
df_transactions_long["Group_id"].value_counts()

24    9
4     9
49    9
46    9
43    9
     ..
63    1
64    1
66    1
67    1
92    1
Name: Group_id, Length: 92, dtype: int64

In [8]:
## saving the transactions in the current working temp directory
df_wide = df_transactions_long.pivot(index='Group_id',columns='Obs',values='Item_barcode')

In [9]:
df_wide.head()

Obs,Inp_item_1,Inp_item_2,Inp_item_3,Inp_item_4,Inp_item_5,Inp_item_6,Inp_item_7,Inp_item_8,Inp_item_9
Group_id,,,,,,,,,
1,5017838815183,5017838815183,5000128725477,5000128725477,5010115926985,NaN,NaN,NaN,NaN
10,5000431027169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,5017838815183,5017838815183,5000128725477,5000128725477,5010115926985,NaN,NaN,NaN,NaN
12,8710732226907,5010891011868,5015552000151,NaN,NaN,NaN,NaN,NaN,NaN
13,7622300754136,5000128797436,5010718402909,5038862320108,NaN,NaN,NaN,NaN,NaN


In [10]:
df_wide.shape

(92, 9)

In [11]:
df_wide.to_csv(r"dataset\df_wide.csv",index=False)

#### Read wide csv format

In [12]:
df_w = pd.read_csv(r"dataset\df_wide.csv",dtype=object)

In [13]:
## parser to convert wide format into list of list
itemlist=[]
for i in range(0,len(df_w)):
    itemlist.append([df_w.iloc[i,j] for j in range(0,df_w.iloc[i].notna().sum())])        

In [14]:
itemlist[15:20]

[['7622300754136', '5000128797436', '5010718402909', '5038862320108'],
 ['5060107333970',
  '5000205039862',
  '5000205034546',
  '5010228013213',
  '8712566328352',
  '4062300011847',
  '5060107331365',
  '5010102240537',
  '5000128889889'],
 ['6191514300312', '5000128715331', '5000128871457', '5000128271165'],
 ['9770307268922', '5000128861069'],
 ['7622210286314']]

In [15]:
## creating the tags for the input and output
def mba_create_tags(X,y):
    glantus_cep_input =[]
    #glantus_cep_output =[]
    
    
    for i in range(1,X+1):
        dic_tag={}
        dic_tag["name"]="Inp_item_"+str(i)
        dic_tag["type"]="string"
        glantus_cep_input.append(dic_tag)
    
        
    glantus_cep_output= [
            {"name": "RecommendedItem", "type": "string"}
        ]
    
    return glantus_cep_input,glantus_cep_output

In [16]:
x=mba_create_tags(50,10)
x

([{'name': 'Inp_item_1', 'type': 'string'},
  {'name': 'Inp_item_2', 'type': 'string'},
  {'name': 'Inp_item_3', 'type': 'string'},
  {'name': 'Inp_item_4', 'type': 'string'},
  {'name': 'Inp_item_5', 'type': 'string'},
  {'name': 'Inp_item_6', 'type': 'string'},
  {'name': 'Inp_item_7', 'type': 'string'},
  {'name': 'Inp_item_8', 'type': 'string'},
  {'name': 'Inp_item_9', 'type': 'string'},
  {'name': 'Inp_item_10', 'type': 'string'},
  {'name': 'Inp_item_11', 'type': 'string'},
  {'name': 'Inp_item_12', 'type': 'string'},
  {'name': 'Inp_item_13', 'type': 'string'},
  {'name': 'Inp_item_14', 'type': 'string'},
  {'name': 'Inp_item_15', 'type': 'string'},
  {'name': 'Inp_item_16', 'type': 'string'},
  {'name': 'Inp_item_17', 'type': 'string'},
  {'name': 'Inp_item_18', 'type': 'string'},
  {'name': 'Inp_item_19', 'type': 'string'},
  {'name': 'Inp_item_20', 'type': 'string'},
  {'name': 'Inp_item_21', 'type': 'string'},
  {'name': 'Inp_item_22', 'type': 'string'},
  {'name': 'Inp_ite

In [17]:
# function for creating list of all the columns
def col_to_list(input_df):
    col_lst=[]
    len_df = len(input_df)
    for i in range(len_df):
        tmp=list(input_df.iloc[i])
        tmp=[x for x in tmp if type(x)==str]
        col_lst.append(tmp)
    return col_lst

In [18]:
def subset(barcode_list):
    listPairs = []
    if len(barcode_list) == 1:
        listPairs.append(barcode_list)
        return listPairs
    for i in range(len(barcode_list)-1):
        for j in range(i+1, len(barcode_list)):
            listPairs.append([barcode_list[i], barcode_list[j]])
    return listPairs

In [19]:
def making_dataframe(listpairs,model,itemlist):
    df_temp = pd.DataFrame(columns = ['FirstItem', 'SecondItem', 'TargetItem', 'Support', 'Confidence'])
    df_intm      = pd.DataFrame(columns = ['RecommendedItem'])
    
    basketItemsList=itemlist
    df = model
    for pair in listpairs:
        df_iter = df[df.FirstItem.isin(pair) & df.SecondItem.isin(pair)]
        df_iter = df_iter[~df_iter.TargetItem.isin(basketItemsList)]
        df_iter = df_iter.sort_values(by=['Support', 'Confidence'], ascending=False)
        df_temp = pd.concat([df_temp, df_iter], ignore_index=True)
#         df_temp = df_temp.append(df_iter, ignore_index=True)
    df_temp = df_temp[["TargetItem", "Support", "Confidence"]]
    df_temp = df_temp.sort_values(by=['Support', 'Confidence'], ascending=False).head(10)
    
    if len(df_temp) != 0: 
        df_intm["RecommendedItem"] = df_temp["TargetItem"]
        df_intm=df_intm.drop_duplicates(subset="RecommendedItem",keep="first")
        return df_intm
    else:
        df_intm["RecommendedItem"] = [""]
        df_intm=df_intm.drop_duplicates(subset="RecommendedItem",keep="first")
        return df_intm

In [20]:
class MBAWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self, model):
        self.model = model

    def predict(self, input_df):
        output_df = pd.DataFrame(columns = ['RecommendedItem'])
        col_list_to_list = col_to_list(input_df)
        for i in col_list_to_list:
            listpairs= subset(i)
            output = making_dataframe(listpairs,self.model,i)
            output_df= pd.concat([output_df,output], axis=0)
        output_df = output_df.fillna('')
        return output_df

In [21]:
## Training of the model function
def train(transactions, support ,confidence):
    itemsets, rules = apriori(transactions, min_support=support,  min_confidence=confidence)
    df_FP_t = pd.DataFrame(columns=['FirstItem', 'SecondItem','TargetItem', 'Support','Confidence'])
    for i in rules:
        
        if len(i.lhs)==2 and len(i.rhs)==1:
            print(i)
            print("______")
            new_row = {
                'FirstItem': i.lhs[0],
                'SecondItem': i.lhs[1],
                'TargetItem': i.rhs[0],
                'Support': i.support,
                'Confidence': i.confidence
            }
            df_FP_t = pd.concat([df_FP_t, pd.DataFrame([new_row])], ignore_index=True)
    df_FP_t["Support"] = round(df_FP_t["Support"], 4)
    df_FP_t["Confidence"] = round(df_FP_t["Confidence"], 4)
    df_FP_t["FirstItem"] = df_FP_t["FirstItem"].replace('', np.nan, regex=True)
    df_FP_t["SecondItem"] = df_FP_t["SecondItem"].replace('', np.nan, regex=True)        
    df_FP_t["TargetItem"] = df_FP_t["TargetItem"].replace('', np.nan, regex=True)        
    df_FP_t=df_FP_t.dropna()
    model=MBAWrapper(df_FP_t)
    glantus_cep_input,glantus_cep_output=mba_create_tags(50,10)
    print("Training Completed")
    return model

In [22]:
model = train(itemlist,0.01, 0.05)

{5000128889889, 5000205034546} -> {4062300011847} (conf: 1.000, supp: 0.076, lift: 13.143, conv: 923913043.478)
______
{4062300011847, 5000205034546} -> {5000128889889} (conf: 1.000, supp: 0.076, lift: 13.143, conv: 923913043.478)
______
{4062300011847, 5000128889889} -> {5000205034546} (conf: 1.000, supp: 0.076, lift: 13.143, conv: 923913043.478)
______
{5000128889889, 5000205039862} -> {4062300011847} (conf: 1.000, supp: 0.076, lift: 13.143, conv: 923913043.478)
______
{4062300011847, 5000205039862} -> {5000128889889} (conf: 1.000, supp: 0.076, lift: 13.143, conv: 923913043.478)
______
{4062300011847, 5000128889889} -> {5000205039862} (conf: 1.000, supp: 0.076, lift: 13.143, conv: 923913043.478)
______
{5000128889889, 5010102240537} -> {4062300011847} (conf: 1.000, supp: 0.076, lift: 13.143, conv: 923913043.478)
______
{4062300011847, 5010102240537} -> {5000128889889} (conf: 1.000, supp: 0.076, lift: 13.143, conv: 923913043.478)
______
{4062300011847, 5000128889889} -> {5010102240537

## Two input baskets

In [23]:
data = [['5010115926985', '5017838815183', '5000128725477','5015552000151', '8710732226907','7622210702005','6191514300312','5000128715331', '','',
        '', '', '','', '', '','', '', '','',
        '', '', '','', '', '','', '', '','',
        '', '', '','', '', '','', '', '','',
        '', '', '','', '', '','', '', '',''],
        ['5010115926985', '5017838815183', '5000128725477','5015552000151', '8710732226907','7622210702005','6191514300312','', '','',
        '', '', '','', '', '','', '', '','',
        '', '', '','', '', '','', '', '','',
        '', '', '','', '', '','', '', '','',
        '', '', '','', '', '','', '', '','']
        ] 

# Create the pandas DataFrame 
input_df = pd.DataFrame(data, columns = ['Inp_item_1', 'Inp_item_2','Inp_item_3', 'Inp_item_4','Inp_item_5', 'Inp_item_6','Inp_item_7', 'Inp_item_8', 'Inp_item_9','Inp_item_10',
                                  'Inp_item_11', 'Inp_item_12','Inp_item_13', 'Inp_item_14','Inp_item_15', 'Inp_item_16','Inp_item_17', 'Inp_item_18', 'Inp_item_19','Inp_item_20',
                                  'Inp_item_21', 'Inp_item_22','Inp_item_23', 'Inp_item_24','Inp_item_25', 'Inp_item_26','Inp_item_27', 'Inp_item_28', 'Inp_item_29','Inp_item_30',
                                  'Inp_item_31', 'Inp_item_32','Inp_item_33', 'Inp_item_34','Inp_item_35', 'Inp_item_36','Inp_item_37', 'Inp_item_38', 'Inp_item_39','Inp_item_40',
                                  'Inp_item_41', 'Inp_item_42','Inp_item_43', 'Inp_item_44','Inp_item_45', 'Inp_item_46','Inp_item_47', 'Inp_item_48', 'Inp_item_49','Inp_item_50'])


input_df.head()

,Inp_item_1,Inp_item_2,Inp_item_3,Inp_item_4,Inp_item_5,Inp_item_6,Inp_item_7,Inp_item_8,Inp_item_9,Inp_item_10,...,Inp_item_41,Inp_item_42,Inp_item_43,Inp_item_44,Inp_item_45,Inp_item_46,Inp_item_47,Inp_item_48,Inp_item_49,Inp_item_50
0,5010115926985,5017838815183,5000128725477,5015552000151,8710732226907,7622210702005,6191514300312,5000128715331,,,...,,,,,,,,,,
1,5010115926985,5017838815183,5000128725477,5015552000151,8710732226907,7622210702005,6191514300312,,,,...,,,,,,,,,,


In [24]:
y = model.predict(input_df)
y

,RecommendedItem
1,5000128271165
2,5000128871457
0,5010891011868
0,5010891011868


In [25]:
x = y.to_json(orient="records")
print(x)

[{"RecommendedItem":"5000128271165"},{"RecommendedItem":"5000128871457"},{"RecommendedItem":"5010891011868"},{"RecommendedItem":"5010891011868"}]


## One basket recommendation

In [26]:
data1 = [['5010115926985', '5010891011868', '','', '','','','', '','',
        '', '', '','', '', '','', '', '','',
        '', '', '','', '', '','', '', '','',
        '', '', '','', '', '','', '', '','',
        '', '', '','', '', '','', '', '','']]

# Create the pandas DataFrame 
input_df1 = pd.DataFrame(data1, columns = ['Inp_item_1', 'Inp_item_2','Inp_item_3', 'Inp_item_4','Inp_item_5', 'Inp_item_6','Inp_item_7', 'Inp_item_8', 'Inp_item_9','Inp_item_10',
                                  'Inp_item_11', 'Inp_item_12','Inp_item_13', 'Inp_item_14','Inp_item_15', 'Inp_item_16','Inp_item_17', 'Inp_item_18', 'Inp_item_19','Inp_item_20',
                                  'Inp_item_21', 'Inp_item_22','Inp_item_23', 'Inp_item_24','Inp_item_25', 'Inp_item_26','Inp_item_27', 'Inp_item_28', 'Inp_item_29','Inp_item_30',
                                  'Inp_item_31', 'Inp_item_32','Inp_item_33', 'Inp_item_34','Inp_item_35', 'Inp_item_36','Inp_item_37', 'Inp_item_38', 'Inp_item_39','Inp_item_40',
                                  'Inp_item_41', 'Inp_item_42','Inp_item_43', 'Inp_item_44','Inp_item_45', 'Inp_item_46','Inp_item_47', 'Inp_item_48', 'Inp_item_49','Inp_item_50'])


input_df1.head()

,Inp_item_1,Inp_item_2,Inp_item_3,Inp_item_4,Inp_item_5,Inp_item_6,Inp_item_7,Inp_item_8,Inp_item_9,Inp_item_10,...,Inp_item_41,Inp_item_42,Inp_item_43,Inp_item_44,Inp_item_45,Inp_item_46,Inp_item_47,Inp_item_48,Inp_item_49,Inp_item_50
0,5010115926985,5010891011868,,,,,,,,,...,,,,,,,,,,


In [27]:
y1 = model.predict(input_df1)
y1

,RecommendedItem
0,


In [28]:
x1 = y1.to_json(orient="records")
print(x1)

[{"RecommendedItem":""}]
